# **Test Case: Skin Disease System**

In [1]:
import os
import numpy as np
import torch
from torch import nn
from torchvision import transforms
from PIL import Image
from transformers import SwinModel, AutoImageProcessor
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

Using device: cuda
GPU: Tesla P100-PCIE-16GB


In [2]:
# Define constants
unique_classes = ['Actinic_keratosis', 'Basal_cell_carcinoma', 'Benign_keratosis', 
                  'Dermatofibroma', 'Melanocytic_nevus', 'Melanoma', 
                  'Squamous_cell_carcinoma', 'Vascular_lesion']
NUM_CLASSES = len(unique_classes)
model_name = "microsoft/swin-large-patch4-window7-224-in22k"

# 1) SWIN Feature Extraction

In [3]:
# Defining the OptimizedSwinClassifier (exact copy from your original code)
class OptimizedSwinClassifier(nn.Module):
    def __init__(self, num_classes=8, dropout_rate=0.3):
        super(OptimizedSwinClassifier, self).__init__()
        self.backbone = SwinModel.from_pretrained(model_name)
        hidden_size = self.backbone.config.hidden_size
        
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.BatchNorm1d(hidden_size // 2),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate // 2),
            nn.Linear(hidden_size // 2, hidden_size // 4),
            nn.BatchNorm1d(hidden_size // 4),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate // 4),
            nn.Linear(hidden_size // 4, num_classes)
        )
    
    def forward(self, pixel_values):
        outputs = self.backbone(pixel_values=pixel_values)
        pooled_output = outputs.last_hidden_state.mean(dim=1)  
        logits = self.classifier(pooled_output)
        return logits
    
    def extract_features(self, pixel_values):
        """Extract the features before the classifier layer (after backbone + pooling)"""
        with torch.no_grad():  
            outputs = self.backbone(pixel_values=pixel_values)
            pooled_output = outputs.last_hidden_state.mean(dim=1)  
            return pooled_output.detach().cpu().numpy()

def load_model(model_path):
    print("Initializing and loading the trained model...")
    model = OptimizedSwinClassifier(num_classes=NUM_CLASSES)
    
    try:
        # Loading the trained weights
        model.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Successfully loaded model from '{model_path}'")
    except FileNotFoundError:
        print(f"Error: Could not find the model file at '{model_path}'")
        print("Please check the file path and ensure the model file exists.")
        raise
    except Exception as e:
        print(f"Error loading model: {e}")
        raise
    
    model.to(device)
    model.eval()  # Set to evaluation mode
    print("Model loaded successfully and set to evaluation mode!")
    return model

def preprocess_image(image_path):
    # Defining the same transforms used during training
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    try:
        # Loading and convert the image to RGB
        img = Image.open(image_path).convert('RGB')
        print(f"Successfully loaded image: {image_path}")
        print(f"Original image size: {img.size}")
        
        # Applying transformations
        img_tensor = transform(img)
        
        # Adding batch dimension
        img_tensor = img_tensor.unsqueeze(0)  # Shape: (1, 3, 224, 224)
        
        return img_tensor
        
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        raise

def extract_image_features(model, image_path):
    """Extract features from a single image"""
    print(f"\nExtracting features from: {image_path}")
    
    # Preprocessing the image
    img_tensor = preprocess_image(image_path)
    
    # Move to device
    img_tensor = img_tensor.to(device)
    
    # Extracting the features
    with torch.no_grad():
        features = model.extract_features(img_tensor)
    
    # Removing the batch dimension since we only have one image
    features = features.squeeze(0)
    
    print(f"Extracted features shape: {features.shape}")
    print(f"Features type: {type(features)}")
    
    return features

def main():
    """Main function to demonstrate usage"""
    
    # Path to the SWIN model
    model_path = "/kaggle/working/final_optimized_swin_model3.pth"
    
    # Load the model
    model = load_model(model_path)
    
    # image path
    image_path = "/kaggle/input/skin-disease-img/Melanocytic_nevus.jpg" 
    
    # Checking if the image exists
    if not os.path.exists(image_path):
        print(f"Error: Image file not found at {image_path}")
        print("Please update the image_path variable with a valid image path")
        return
    
    try:
        # Extracting the features
        features = extract_image_features(model, image_path)
        
        print(f"\n" + "="*50)
        print("FEATURE EXTRACTION COMPLETED")
        print("="*50)
        print(f"Features shape: {features.shape}")
        print(f"Features dtype: {features.dtype}")
        print(f"First 10 feature values: {features[:10]}")
        return features
        
    except Exception as e:
        print(f"Error during feature extraction: {e}")
        return None

def extract_features_batch(model, image_paths, batch_size=32):
    """Extract features from multiple images"""
    all_features = []
    
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_tensors = []
        
        for img_path in batch_paths:
            try:
                img_tensor = preprocess_image(img_path)
                batch_tensors.append(img_tensor.squeeze(0))  # Remove batch dim for stacking
            except Exception as e:
                print(f"Skipping {img_path} due to error: {e}")
                continue
        
        if batch_tensors:
            # Stacking into batch
            batch_tensor = torch.stack(batch_tensors).to(device)
            
            # Extracting the features
            with torch.no_grad():
                batch_features = model.extract_features(batch_tensor)
            
            all_features.append(batch_features)
            
            # Clear the GPU cache periodically
            if i % (batch_size * 10) == 0:
                torch.cuda.empty_cache()
    
    if all_features:
        return np.concatenate(all_features, axis=0)
    else:
        return np.array([])

if __name__ == "__main__":
    main()

Initializing and loading the trained model...


config.json:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/915M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/915M [00:00<?, ?B/s]

Successfully loaded model from '/kaggle/working/final_optimized_swin_model3.pth'
Model loaded successfully and set to evaluation mode!

Extracting features from: /kaggle/input/skin-disease-img/Melanoma.jpg
Successfully loaded image: /kaggle/input/skin-disease-img/Melanoma.jpg
Original image size: (250, 174)
Extracted features shape: (1536,)
Features type: <class 'numpy.ndarray'>

FEATURE EXTRACTION COMPLETED
Features shape: (1536,)
Features dtype: float32
First 10 feature values: [-0.11700546 -0.09721308  0.21264325 -0.1347722   0.31371462  0.23774213
 -0.21733057 -0.12520638 -0.24066457 -0.04410677]


# 2) RF Model

In [4]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import LabelEncoder

In [5]:
# Loading the Random Forest model
def load_rf_model(model_path):
    """Load the trained Random Forest model"""
    try:
        print(f"Loading Random Forest model from: {model_path}")
        rf_model = joblib.load(model_path)
        print("Random Forest model loaded successfully!")
        return rf_model
    except Exception as e:
        print(f"Error loading Random Forest model: {e}")
        raise


In [6]:
# Creating the user metadata input structure
def create_user_metadata_input(age, gender, anatomic_site):
    # Defining all the possible anatomic sites from the training data
    anatomic_sites = [
        'anterior torso', 'head/neck', 'lower extremity', 
        'oral/genital', 'palms/soles', 'posterior torso', 'upper extremity'
    ]
    
    # Defining the age ranges
    age_ranges = [
        '1-10', '11-20', '21-30', '31-40', 
        '41-50', '51-60', '61-70', '71-90'
    ]
    
    # Validating the inputs
    if gender.lower() not in ['male', 'female']:
        raise ValueError("Gender must be 'male' or 'female'")
    
    if anatomic_site not in anatomic_sites:
        raise ValueError(f"Anatomic site must be one of: {anatomic_sites}")
    
    # Determining the age range
    if age <= 10:
        age_range = '1-10'
    elif age <= 20:
        age_range = '11-20'
    elif age <= 30:
        age_range = '21-30'
    elif age <= 40:
        age_range = '31-40'
    elif age <= 50:
        age_range = '41-50'
    elif age <= 60:
        age_range = '51-60'
    elif age <= 70:
        age_range = '61-70'
    else:
        age_range = '71-90'
    
    # Creating the metadata dictionary
    metadata = {}
    
    # Initializing all the anatomic site columns to 0
    for site in anatomic_sites:
        metadata[f'site_{site}'] = 0
    
    # Seting the user's anatomic site to 1
    metadata[f'site_{anatomic_site}'] = 1
    
    # Initializ ing all the gender columns to 0
    metadata['sex_female'] = 0
    metadata['sex_male'] = 0
    
    # Seting the user's gender to 1
    if gender.lower() == 'female':
        metadata['sex_female'] = 1
    else:
        metadata['sex_male'] = 1
    
    # Initializing all the  age range columns to 0
    for age_r in age_ranges:
        metadata[f'age_{age_r}'] = 0
    
    # Seting the user's age range to 1
    metadata[f'age_{age_range}'] = 1
    
    # Converting to DataFrame
    metadata_df = pd.DataFrame([metadata])
    
    print(f"Created metadata for:")
    print(f"  Age: {age} (range: {age_range})")
    print(f"  Gender: {gender}")
    print(f"  Anatomic site: {anatomic_site}")
    
    return metadata_df

# Combining the image features with the user metadata
def combine_features_metadata(image_features, user_metadata_df):    
    # Convert image features to DataFrame
    feature_cols = [f"feature_{i}" for i in range(len(image_features))]
    features_df = pd.DataFrame([image_features], columns=feature_cols)
    
    # Combining the features with the metadata
    combined_input = pd.concat([user_metadata_df.reset_index(drop=True), 
                               features_df.reset_index(drop=True)], axis=1)
    
    print(f"Combined input shape: {combined_input.shape}")
    print(f"Image features: {len(image_features)} dimensions")
    print(f"Metadata features: {user_metadata_df.shape[1]} dimensions")
    
    return combined_input

# Making the prediction by using the Random Forest model
def predict_disease(rf_model, combined_input):    
    try:
        # Making prediction
        prediction = rf_model.predict(combined_input)
        prediction_proba = rf_model.predict_proba(combined_input)
        
        # Geting the class names
        class_names = rf_model.classes_
        
        # Geting the predicted disease
        predicted_disease = prediction[0]
        
        # Geting the prediction probabilities
        prob_dict = {class_names[i]: prediction_proba[0][i] for i in range(len(class_names))}
        
        print(f"\nPrediction Results:")
        print("=" * 40)
        print(f"Predicted Disease: {predicted_disease}")
        print(f"Confidence: {prob_dict[predicted_disease]:.4f}")
        
        print(f"\nAll Class Probabilities:")
        print("-" * 30)
        for disease, prob in sorted(prob_dict.items(), key=lambda x: x[1], reverse=True):
            print(f"{disease}: {prob:.4f}")
        
        return predicted_disease
        
    except Exception as e:
        print(f"Error making prediction: {e}")
        raise

# The Main pipeline function
def run_prediction_pipeline(rf_model_path, image_features, age, gender, anatomic_site):    
    print("=" * 60)
    print("RUNNING DISEASE PREDICTION PIPELINE")
    print("=" * 60)
    
    # Loading the RF model
    rf_model = load_rf_model(rf_model_path)
    
    # Creating the user metadata
    user_metadata = create_user_metadata_input(age, gender, anatomic_site)
    
    # Combining the features with the metadata
    combined_input = combine_features_metadata(image_features, user_metadata)
    
    # Making the prediction
    predicted_disease = predict_disease(rf_model, combined_input)
    
    print("=" * 60)
    print("PIPELINE COMPLETED SUCCESSFULLY")
    print("=" * 60)
    
    return predicted_disease

def example_usage():  
    
    rf_model_path = "/kaggle/input/rf_best_model/tensorflow2/default/1/best_rf_model.pkl"
    
    example_features = np.random.randn(1536)  
    
    # Example of the user metadata 
    user_age = 45
    user_gender = "female" 
    user_anatomic_site = "anterior torso"  
    
    # The anatomic sites:
    available_sites = [
        'anterior torso', 'head/neck', 'lower extremity', 
        'oral/genital', 'palms/soles', 'posterior torso', 'upper extremity'
    ]
    
    print("Available anatomic sites:")
    for i, site in enumerate(available_sites, 1):
        print(f"  {i}. {site}")
    
    try:
        # Runing the prediction pipeline
        predicted_disease = run_prediction_pipeline(
            rf_model_path=rf_model_path,
            image_features=example_features,
            age=user_age,
            gender=user_gender,
            anatomic_site=user_anatomic_site
        )
        
        print(f"\nFinal Result: The predicted disease is '{predicted_disease}'")
        return predicted_disease
        
    except Exception as e:
        print(f"Error in prediction pipeline: {e}")
        return None

# A function to get the user input interactively (But I didn't used it in this test case)
def get_user_input():  
    print("Please provide the following information:")
    print("-" * 40)
    
    # Get age
    while True:
        try:
            age = int(input("Enter patient age (1-90): "))
            if 1 <= age <= 90:
                break
            else:
                print("Age must be between 1 and 90")
        except ValueError:
            print("Please enter a valid number")
    
    # Geting the gender
    while True:
        gender = input("Enter patient gender (male/female): ").lower().strip()
        if gender in ['male', 'female']:
            break
        else:
            print("Please enter 'male' or 'female'")
    
    # Geting the anatomic site
    sites = [
        'anterior torso', 'head/neck', 'lower extremity', 
        'oral/genital', 'palms/soles', 'posterior torso', 'upper extremity'
    ]
    
    print("\nAvailable anatomic sites:")
    for i, site in enumerate(sites, 1):
        print(f"  {i}. {site}")
    
    while True:
        try:
            site_choice = int(input("Enter anatomic site number (1-7): "))
            if 1 <= site_choice <= 7:
                anatomic_site = sites[site_choice - 1]
                break
            else:
                print("Please enter a number between 1 and 7")
        except ValueError:
            print("Please enter a valid number")
    
    return age, gender, anatomic_site

if __name__ == "__main__":
    # Runing the example
    example_usage()

Available anatomic sites:
  1. anterior torso
  2. head/neck
  3. lower extremity
  4. oral/genital
  5. palms/soles
  6. posterior torso
  7. upper extremity
RUNNING DISEASE PREDICTION PIPELINE
Loading Random Forest model from: /kaggle/input/rf_best_model/tensorflow2/default/1/best_rf_model.pkl
Random Forest model loaded successfully!
Created metadata for:
  Age: 45 (range: 41-50)
  Gender: female
  Anatomic site: anterior torso
Combined input shape: (1, 1553)
Image features: 1536 dimensions
Metadata features: 17 dimensions

Prediction Results:
Predicted Disease: Melanocytic_nevus
Confidence: 0.1619

All Class Probabilities:
------------------------------
Melanocytic_nevus: 0.1619
Squamous_cell_carcinoma: 0.1595
Actinic_keratosis: 0.1430
Dermatofibroma: 0.1216
Benign_keratosis: 0.1173
Vascular_lesion: 0.1117
Melanoma: 0.0958
Basal_cell_carcinoma: 0.0892
PIPELINE COMPLETED SUCCESSFULLY

Final Result: The predicted disease is 'Melanocytic_nevus'


# 3) Multi Agent System 

In [7]:
# Install required packages if not already installed
!pip install --upgrade langchain langgraph langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
  Attempting uninstall: langsmith
   

In [8]:
import os
from langchain.schema import SystemMessage, HumanMessage
from langgraph.graph import StateGraph, END
from typing import TypedDict
import re
from langchain_openai import ChatOpenAI

In [17]:
os.environ['OPENROUTER_API_KEY'] = "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"  # <-- Replace with your key

class MedicalState(TypedDict):
    disease: str
    country: str
    disease_info: str
    hospital_list: str

class AgentMemory:
    def __init__(self):
        self.chat_log = []
    
    def update(self, message: str):
        self.chat_log.append(message)
    
    def get_context(self, n=5):
        return "\n".join(self.chat_log[-n:])

memory = AgentMemory()


def get_llm(model: str = "gpt-3.5-turbo", temp: float = 0.2):
    return ChatOpenAI(
        openai_api_key=os.environ['OPENROUTER_API_KEY'],
        openai_api_base="https://openrouter.ai/api/v1",
        model=model,
        temperature=temp,
        max_retries=3,
        timeout=30
    )

def disease_info_agent(state: MedicalState) -> MedicalState:
    prompt = f"""You are a helpful medical assistant.
Provide a clear and concise 4–5 sentence description of the skin disease "{state['disease']}" and after that only mention the following briefly in this format: 
1- its symptoms: 
2- its causes:
3- the possible treatments:.

Disease: {state['disease']}
Description:"""

    llm = get_llm("gpt-3.5-turbo", 0.2)
    response = llm.invoke([
        SystemMessage(content="You are a reliable, medically informed assistant."),
        HumanMessage(content=prompt)
    ]).content
    
    memory.update(f"Disease Info: {response}")
    return {**state, "disease_info": response}

def hospital_finder_agent(state: MedicalState) -> MedicalState:
    prompt = f"""List 5 well-known hospitals or dermatology clinics in {state['country']} where someone can go for treatment of a skin disease.
Include their:
- name
- city
- contact number (if possible)
- a short address

Only show relevant institutions with a reputation for dermatology or skin care."""

    llm = get_llm("gpt-3.5-turbo", 0.2)
    response = llm.invoke([
        SystemMessage(content="You are a medical travel assistant helping people find hospitals."),
        HumanMessage(content=prompt)
    ]).content
    
    memory.update(f"Hospital Info: {response}")
    return {**state, "hospital_list": response}

def build_workflow():
    graph = StateGraph(MedicalState)
    graph.add_node("describe_disease", disease_info_agent)
    graph.add_node("find_hospitals", hospital_finder_agent)
    graph.set_entry_point("describe_disease")
    graph.add_edge("describe_disease", "find_hospitals")
    graph.add_edge("find_hospitals", END)
    return graph.compile()

def medical_assistant(disease: str, country: str):
    initial_state = {
        "disease": disease.strip(),
        "country": country.strip(),
        "disease_info": "",
        "hospital_list": ""
    }
    
    try:
        assistant = build_workflow()
        final_state = assistant.invoke(initial_state)

        output = f"""
==============================
Information About: {final_state['disease']}
==============================
{final_state['disease_info']}

==============================
Hospitals in {final_state['country']}
==============================
{final_state['hospital_list']}
"""
        return output

    except Exception as e:
        return f"An error occurred: {str(e)}"



# Giving it the Diagnosed Disease (the Melanocytic_nevus)
if __name__ == "__main__":
    disease = 'Melanocytic_nevus'
    country = input("Enter your country to find hospitals: ")

    print("\nRunning Medical Assistant...\n")
    result = medical_assistant(disease, country)
    print(result)

Enter your country to find hospitals:  England



Running Medical Assistant...


Information About: Melanocytic_nevus
Melanocytic nevus, commonly known as a mole, is a benign skin growth that develops when melanocytes (cells that produce pigment) grow in clusters. These moles can vary in color, size, and shape, and are usually harmless. However, they should be monitored for any changes in appearance that could indicate skin cancer.

Symptoms: Typically, melanocytic nevi appear as small, dark brown spots on the skin. They can be flat or raised, and may have a smooth or rough texture.

Causes: The exact cause of melanocytic nevi is not fully understood, but they are believed to be influenced by genetic factors and sun exposure.

Possible treatments: In most cases, no treatment is needed for melanocytic nevi. However, if a mole shows signs of change or is at risk of becoming cancerous, it may need to be removed surgically. Regular skin checks and monitoring by a dermatologist are recommended for individuals with a large number of moles 